This is a transformer pratice for Apr22. I have tried Bi-LSTM for this playground which performance is better than this one. My opion is the dataset is too small to train transformer model. If you have another ideas. Please leave comments! ;)

In [ ]:
import pandas as pd
from matplotlib import pyplot as plt

In [ ]:
train = pd.read_csv('../input/tabular-playground-series-apr-2022/train.csv')
train_labels = pd.read_csv('../input/tabular-playground-series-apr-2022/train_labels.csv')
test = pd.read_csv('../input/tabular-playground-series-apr-2022/test.csv')

## Feature Engineering

In [ ]:
def add_lag_features(df):

    sensors = ['sensor_'+'%02d'%i for i in range(0, 13)]
    for sensor in sensors:
        df[sensor + '_lag1'] = df.groupby('sequence')[sensor].shift(1)
        df.fillna(0, inplace=True)
        df[sensor + '_diff1'] = df[sensor] - df[sensor + '_lag1']

    return df

In [ ]:
train = add_lag_features(train)
test = add_lag_features(test)

In [ ]:
Window = 60
y = train_labels['state'].to_numpy()
train.drop(["sequence","step","subject"], axis=1, inplace=True)
test.drop(["sequence","step","subject"], axis=1, inplace=True)


In [ ]:
train

## Feature Normalization

In [ ]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
sc.fit(train)
X_train = sc.transform(train)
X_test = sc.transform(test)

## Reshape Data

In [ ]:
y_train = y.copy()
X_train = X_train.reshape(-1, Window, X_train.shape[-1])
X_test = X_test.reshape(-1, Window, X_test.shape[-1])

In [ ]:
print(y_train.shape, X_train.shape, X_test.shape)

## Build Transformer Model

In [ ]:
from keras import Model, layers
from keras.layers import *
from keras.callbacks import *
from keras.metrics import AUC

In this note, I use encoder part as seq2seq model.
![encoder](https://uvadlc-notebooks.readthedocs.io/en/latest/_images/transformer_architecture.svg)

In [ ]:
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0.0):
    # Attention and Normalization
    x = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(inputs, inputs)
    x = layers.Dropout(dropout)(x)
    x = layers.LayerNormalization(epsilon=1e-6)(x)
#   residual and add norm
    res = x + inputs 

    # Feed Forward Part
    x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="selu")(res)
    x = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    x = layers.LayerNormalization(epsilon=1e-6)(x)
#   residual and add norm
    return x + res

In [ ]:
def build_model(
    shape,
    head_size = 128,
    num_heads = 8,
    ff_dim=32,
    num_transformer_blocks=4,
    mlp_units=[128],
    dropout=0.0,
    mlp_dropout=0.4,
):
    inputs = Input(shape=shape)
    x = inputs
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

    x = layers.GlobalAveragePooling1D()(x)
    for dim in mlp_units:
        x = Dense(dim, activation="selu")(x)
        x = Dropout(mlp_dropout)(x)

    outputs = Dense(1, activation='sigmoid')(x)

    model = Model(inputs=inputs, outputs=outputs, name='lstm_model')

    model.compile(optimizer='adam',
                      loss='binary_crossentropy',
                      metrics=[AUC(name = 'auc')])
    return  model

In [ ]:
model = build_model(shape=X_train.shape[1:])
model.summary()

In [ ]:
from keras.utils.vis_utils import plot_model
plot_model(model, show_shapes=True)

In [ ]:
X_val = X_train[23372:]
partial_x_train = X_train[:23372]
Y_val = y_train[23372:]
partial_y_train = y_train[:23372]

In [ ]:
lr = ReduceLROnPlateau(monitor="val_auc", mode='max', factor=0.7, patience=4, verbose=False)
es = EarlyStopping(monitor='val_auc',mode='max', patience=10, verbose=False,restore_best_weights=True)
history = model.fit(partial_x_train,
            partial_y_train,
            epochs=200,
            batch_size=128,
            validation_data=(X_val, Y_val),
            callbacks=[es,lr]
          )

In [ ]:
def plot_history(hist, metric='auc'):
    plt.plot(hist.history[metric])
    plt.plot(hist.history["val_" + metric])
    plt.title("model performance")
    plt.ylabel("area_under_curve")
    plt.xlabel("epoch")
    plt.legend(["train", "validation"], loc="upper left")


In [ ]:
plot_history(history)

## Submit result

In [ ]:
sub_data = pd.read_csv("../input/tabular-playground-series-apr-2022/sample_submission.csv")

In [ ]:
sub_data['state'] = model.predict(X_test).squeeze()

In [ ]:
sub_data

In [ ]:
sub_data.to_csv('submission.csv', index=False)